这里我们将对CAMB中的fortran源码进行重构,并全部用python实现

我们关注的是scalar mode下的Cl理论值

首先我们取出源码中计算Cl的部分
```fortran
subroutine CalcScalCls(CTrans)
    use Bispectrum
    implicit none
    Type(ClTransferData) :: CTrans
    integer j, q_ix, w_ix, w_ix2
    real(dl) apowers
    real(dl) dlnk, ell, ctnorm, dbletmp, Delta1, Delta2
    real(dl), allocatable :: ks(:), dlnks(:), pows(:)
    real(dl) fac(3 + State%num_redshiftwindows + State%CP%CustomSources%num_custom_sources)
    integer nscal, i

    allocate(ks(CTrans%q%npoints),dlnks(CTrans%q%npoints), pows(CTrans%q%npoints))
    do q_ix = 1, CTrans%q%npoints
        if (State%flat) then
            ks(q_ix) = CTrans%q%points(q_ix)
            dlnks(q_ix) = CTrans%q%dpoints(q_ix)/CTrans%q%points(q_ix)
        else
            ks(q_ix) = sqrt(CTrans%q%points(q_ix)**2 - State%curv)
            dlnks(q_ix) = CTrans%q%dpoints(q_ix)*CTrans%q%points(q_ix)/ks(q_ix)**2
        end if
        pows(q_ix) = CP%InitPower%ScalarPower(ks(q_ix))
        if (global_error_flag/=0) return
    end do

#ifndef __INTEL_COMPILER
    !Can't use OpenMP here on ifort. Does not terminate.
    !$OMP PARALLEL DO DEFAULT(SHARED), SCHEDULE(STATIC,4), &
    !$OMP PRIVATE(ell,q_ix,dlnk,apowers,ctnorm,dbletmp,Delta1,Delta2,w_ix,w_ix2,fac, nscal, i)
#endif
    do j=1,CTrans%ls%nl
        !Integrate dk/k Delta_l_q**2 * Power(k)
        ell = real(CTrans%ls%l(j),dl)
        if (j<= CTrans%max_index_nonlimber) then
            do q_ix = 1, CTrans%q%npoints
                if (.not.(State%closed.and.nint(CTrans%q%points(q_ix)*State%curvature_radius)<=CTrans%ls%l(j))) then
                    !cut off at nu = l + 1
                    dlnk = dlnks(q_ix)
                    apowers = pows(q_ix)

                    iCl_scalar(j,C_Temp:C_E) = iCl_scalar(j,C_Temp:C_E) +  &
                        apowers*CTrans%Delta_p_l_k(1:2,j,q_ix)**2*dlnk
                    iCl_scalar(j,C_Cross) = iCl_scalar(j,C_Cross) + &
                        apowers*CTrans%Delta_p_l_k(1,j,q_ix)*CTrans%Delta_p_l_k(2,j,q_ix)*dlnk

                    if (CTrans%NumSources>2 .and. State%CP%want_cl_2D_array) then

                        do w_ix=1,3 + State%num_redshiftwindows
                            Delta1= CTrans%Delta_p_l_k(w_ix,j,q_ix)
                            if (w_ix>3) then
                                associate (Win => State%Redshift_w(w_ix - 3))
                                    if (Win%kind == window_lensing) &
                                        Delta1 = Delta1 / 2 * ell * (ell + 1)
                                    if (Win%kind == window_counts .and. CP%SourceTerms%counts_lensing) then
                                        !want delta f/f - 2kappa;
                                        ! grad^2 = -l(l+1);
                                        Delta1 = Delta1 + ell * (ell + 1) * &
                                            CTrans%Delta_p_l_k(3 + Win%mag_index + &
                                            State%num_redshiftwindows, j, q_ix)
                                    end if
                                end associate
                            end if
                            do w_ix2=w_ix,3 + State%num_redshiftwindows
                                if (w_ix2>= 3.and. w_ix>=3) then
                                    !Skip if the auto or cross-correlation is included in direct Limber result
                                    !Otherwise we need to include the sources e.g. to get counts-Temperature correct
                                    if (CTrans%limber_l_min(w_ix2)/= 0 .and. j>=CTrans%limber_l_min(w_ix2) &
                                        .and. CTrans%limber_l_min(w_ix)/= 0 .and. j>=CTrans%limber_l_min(w_ix)) cycle

                                end if
                                Delta2 = CTrans%Delta_p_l_k(w_ix2, j, q_ix)
                                if (w_ix2 > 3) then
                                    associate (Win => State%Redshift_w(w_ix2 - 3))
                                        if (Win%kind == window_lensing) &
                                            Delta2 = Delta2 / 2 * ell * (ell + 1)
                                        if (Win%kind == window_counts .and. CP%SourceTerms%counts_lensing) then
                                            !want delta f/f - 2kappa;
                                            ! grad^2 = -l(l+1);
                                            Delta2 = Delta2 + ell * (ell + 1) * &
                                                CTrans%Delta_p_l_k(3 + Win%mag_index + &
                                                State%num_redshiftwindows, j, q_ix)
                                        end if
                                    end associate
                                end if
                                iCl_Array(j,w_ix,w_ix2) = iCl_Array(j,w_ix,w_ix2)+Delta1*Delta2*apowers*dlnk
                            end do
                        end do
                        if (CP%CustomSources%num_custom_sources >0) then
                            do w_ix=1,3 + State%num_redshiftwindows + CP%CustomSources%num_custom_sources
                                if (w_ix > 3 + State%num_redshiftwindows) then
                                    Delta1= CTrans%Delta_p_l_k(w_ix+State%num_extra_redshiftwindows,j,q_ix)
                                else
                                    Delta1= CTrans%Delta_p_l_k(w_ix,j,q_ix)
                                end if
                                do w_ix2=max(w_ix,3 + State%num_redshiftwindows +1), &
                                    3 + State%num_redshiftwindows +CP%CustomSources%num_custom_sources
                                    Delta2=  CTrans%Delta_p_l_k(w_ix2+State%num_extra_redshiftwindows,j,q_ix)
                                    iCl_Array(j,w_ix,w_ix2) = iCl_Array(j,w_ix,w_ix2) &
                                        +Delta1*Delta2*apowers*dlnk
                                end do
                            end do
                        end if
                    end if

                    if (CTrans%NumSources>2 ) then
                        if (CP%SourceTerms%limber_phi_lmin==0 .or.  &
                            CTrans%limber_l_min(3)== 0 .or. j<CTrans%limber_l_min(3)) then
                            iCl_scalar(j,C_Phi) = iCl_scalar(j,C_Phi) +  &
                                apowers*CTrans%Delta_p_l_k(3,j,q_ix)**2*dlnk
                            iCl_scalar(j,C_PhiTemp) = iCl_scalar(j,C_PhiTemp) +  &
                                apowers*CTrans%Delta_p_l_k(3,j,q_ix)*CTrans%Delta_p_l_k(1,j,q_ix)*dlnk
                            iCl_scalar(j,C_PhiE) = iCl_scalar(j,C_PhiE) +  &
                                apowers*CTrans%Delta_p_l_k(3,j,q_ix)*CTrans%Delta_p_l_k(2,j,q_ix)*dlnk
                        end if
                    end if
                end if
            end do

        end if !limber (j<= max_bessels_l_index)

        !Output l(l+1)C_l/OutputDenominator
        ctnorm=(ell*ell-1)*(ell+2)*ell
        dbletmp=(ell*(ell+1))/OutputDenominator*const_fourpi
        if (State%CP%want_cl_2D_array) then
            fac=1
            fac(2) = sqrt(ctnorm)
            if (CTrans%NumSources > 2) then
                fac(3) = sqrt(ell*(ell+1)*CP%ALens) !Changed Dec18 for consistency
                do w_ix=3 + State%num_redshiftwindows+1,3 + State%num_redshiftwindows + CP%CustomSources%num_custom_sources
                    nscal= CP%CustomSources%custom_source_ell_scales(w_ix - State%num_redshiftwindows -3)
                    do i=1, nscal
                        fac(w_ix) = fac(w_ix)*(ell+i)*(ell-i+1)
                    end do
                    fac(w_ix) = sqrt(fac(w_ix))
                end do
            end if

            do w_ix=1, CTrans%NumSources - State%num_extra_redshiftwindows
                do w_ix2=w_ix,CTrans%NumSources - State%num_extra_redshiftwindows
                    iCl_Array(j,w_ix,w_ix2) =iCl_Array(j,w_ix,w_ix2) &
                        *fac(w_ix)*fac(w_ix2)*dbletmp
                    iCl_Array(j,w_ix2,w_ix) = iCl_Array(j,w_ix,w_ix2)
                end do
            end do
        end if

        iCl_scalar(j,C_Temp)  =  iCl_scalar(j,C_Temp)*dbletmp
        iCl_scalar(j,C_E) =  iCl_scalar(j,C_E)*dbletmp*ctnorm
        iCl_scalar(j,C_Cross) =  iCl_scalar(j,C_Cross)*dbletmp*sqrt(ctnorm)
        if (CTrans%NumSources>2) then
            iCl_scalar(j,C_Phi) = CP%ALens*iCl_scalar(j,C_Phi)*const_fourpi*ell**4
            !The lensing power spectrum computed is l^4 C_l^{\phi\phi}
            !We put pix extra factors of l here to improve interpolation in l
            iCl_scalar(j,C_PhiTemp) = sqrt(CP%ALens)*  iCl_scalar(j,C_PhiTemp)*const_fourpi*ell**3
            !Cross-correlation is CTrans%ls%l^3 C_l^{\phi T}
            iCl_scalar(j,C_PhiE) = sqrt(CP%ALens)*  iCl_scalar(j,C_PhiE)*const_fourpi*ell**3*sqrt(ctnorm)
            !Cross-correlation is CTrans%ls%l^3 C_l^{\phi E}
        end if
    end do
#ifndef __INTEL_COMPILER
    !$OMP END PARALLEL DO
#endif

    end subroutine CalcScalCls
```
我们需要的是TT,TE与EE,camb并没有提供low-ell-EB与BB部分的计算

接下来我们将对这部分代码做python实现

In [ ]:
# 导入包
import numpy as np
import matplotlib.pyplot as plt
import scipy

from astropy.constants import c
const_c = c.to('km/s').value

In [ ]:
class ClTransferData:
    def __init__(self):
        # general
        self.NumSources = 2 # 2 for just CMB, 3 for lensing
        self.Delta_p_l_k = np.array([0.0, 0.0, 0.0])
        # self.Limber_l_min = np.array([0.0])
        # self.max_index_nonlimber = 
        # ls
        # self.nl = 0
        # self.lmin = 2
        # self.l = np.array([0.0])
        # self.use_spline_template = True
        # p(Trange) 
        # npoints -> k_max

# input parameter
CTrancs = ClTransferData() 

In [ ]:
# (flat) initial power spectrum
# from planck 2018
def Pk(k):
    As = 2.092e-9
    ns = 0.9647
    n_run = 0.0011
    n_runrun = 0.009
    k0 = 0.05

    lnrat = np.log(k / k0)
    return As * np.exp(lnrat * (ns - 1 + lnrat * (n_run / 2 + lnrat * n_runrun / 6)))

这里只计算CMB

In [ ]:
# integrate dk/k * Delta_l_q ** 2 * P(k) 
def iCl_scalar(l, method):
    k_max = np.inf
    if method == 'TT':
        def integrand(k, l):
            return (Delta_p_l_k[1, l, k] ** 2 * Pk(k) / k)
        return scipy.integrate.quad(integrand, 0, k_max, args=(l))[0]
    elif method == 'EE':
        def integrand(k, l):
            return (Delta_p_l_k[2, l, k] ** 2 * Pk(k) / k)
        return scipy.integrate.quad(integrand, 0, k_max, args=(l))[0]
    elif method == 'TE':
        def integrand(k, l):
            return (Delta_p_l_k[1, l, k] * Delta_p_l_k[2, l, k] * Pk(k) / k)
        return scipy.integrate.quad(integrand, 0, k_max, args=(l))[0]
    else:
        print('method not supported')
        return 0

In [ ]:
# output l(l+1)Cl/2pi
def Cl_scalar(l, method):
    ctnorm = (l * l - 1) * (l + 2) * l
    dbletmp = (l * (l + 1)) * 2
    if method == 'TT':
        return iCl_scalar(l, method) * dbletmp
    elif method == 'EE':
        return iCl_scalar(l, method) * ctnorm * dbletmp
    elif method == 'TE':
        return iCl_scalar(l, method) * np.sqrt(ctnorm) * dbletmp
    else:
        print('method not supported')
        return 0

```fortran
subroutine TimeSourcesToCl(ThisCT)
Type(ClTransferData) :: ThisCT 
integer q_ix
Type(TTimer) :: Timer

if (CP%WantScalars) ThisSources => State%ScalarTimeSources

if (DebugMsgs .and. Feedbacklevel > 0) call Timer%Start()

if (CP%WantScalars .and. WantLateTime &
    .and. (CP%NonLinear==NonLinear_Lens .or. CP%NonLinear==NonLinear_both) .and. global_error_flag==0) then
    call MakeNonlinearSources
    if (DebugMsgs .and. Feedbacklevel > 0) call Timer%WriteTime('Timing for NonLinear sources')
else
    ScaledSrc => ThisSources%LinearSrc
end if

if (global_error_flag==0) then
    call InitSourceInterpolation

    ExactClosedSum = State%curv > 5e-9_dl .or. State%scale < 0.93_dl

    max_bessels_l_index = ThisCT%ls%nl
    max_bessels_etak  = maximum_qeta

    if (CP%WantScalars) call GetLimberTransfers(ThisCT)
    ThisCT%max_index_nonlimber = max_bessels_l_index

    if (State%flat) call InitSpherBessels(ThisCT%ls, CP, max_bessels_l_index,max_bessels_etak )
    !This is only slow if not called before with same (or higher) Max_l, Max_eta_k
    !Preferably stick to Max_l being a multiple of 50

    call SetkValuesForInt(ThisCT)

    if (DebugMsgs .and. Feedbacklevel > 0) call WriteFormat('Set %d integration k values',ThisCT%q%npoints)

    !Begin k-loop and integrate Sources*Bessels over time
    !$OMP PARALLEL DO DEFAULT(SHARED), SCHEDULE(STATIC,4)
    do q_ix=1,ThisCT%q%npoints
        call SourceToTransfers(ThisCT, q_ix)
    end do !q loop
    !$OMP END PARALLEL DO

    if (DebugMsgs .and. Feedbacklevel > 0) call Timer%WriteTime('Timing for Integration')
end if

if (allocated(ddScaledSrc)) deallocate(ddScaledSrc)
if (associated(ScaledSrc) .and. .not. associated(ScaledSrc,ThisSources%LinearSrc)) then
    deallocate(ScaledSrc)
    nullify(ScaledSrc)
end if

!Final calculations for CMB output unless want the Cl transfer functions only.
if (.not. State%OnlyTransfer .and. global_error_flag==0) &
    call ClTransferToCl(State)

if (DebugMsgs .and. Feedbacklevel > 0) call Timer%WriteTime('Timing for final CL output')

end subroutine TimeSourcesToCl
```

使用linear model  

In [ ]:
# skip init

IDM模型为flat